# Stock NeurIPS2018 Part 1. Data
This series is a reproduction of paper *the process in the paper Practical Deep Reinforcement Learning Approach for Stock Trading*.

This is the first part of the NeurIPS2018 series, introducing how to use FinRL to fetch and process data that we need for ML/RL trading.

Other demos can be found at the repo of [FinRL-Tutorials]((https://github.com/AI4Finance-Foundation/FinRL-Tutorials)).

# Part 1. Install Packages

In [1]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
## install finrl library
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.5 MB/s eta 0:00:00
  Created wheel for ecos: filename=ecos-2.0.14-cp312-cp312-macosx_11_0_arm64.whl size=72517 sha256=94d46ffa2c50592ebce188ee8ca80738b9b1cea8956902ec9a7e4fedcd6e6c64
  Stored in directory: /Users/gary/Library/Caches/pip/wheels/b8/d9/1e/77a592ed2465d4dde07f30f787f9d43834c0fd7a1467371694
Successfully built ecos
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-ks3d3w3f
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /private/var/folders/ks/bjl76g8d4zxgw0m5p8z2pd9r0000gn/T/pip-req-build-ks3d3w3f
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit 

In [2]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

! pip install pandas_market_calendars
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

# Part 2. Fetch data

[yfinance](https://github.com/ranaroussi/yfinance) is an open-source library that provides APIs fetching historical data form Yahoo Finance. In FinRL, we have a class called [YahooDownloader](https://github.com/AI4Finance-Foundation/FinRL/blob/master/finrl/meta/preprocessor/yahoodownloader.py) that use yfinance to fetch data from Yahoo Finance.

**OHLCV**: Data downloaded are in the form of OHLCV, corresponding to **open, high, low, close, volume,** respectively. OHLCV is important because they contain most of numerical information of a stock in time series. From OHLCV, traders can get further judgement and prediction like the momentum, people's interest, market trends, etc.

## Data for a single ticker

Here we provide two ways to fetch data with single ticker, let's take Apple Inc. (AAPL) as an example.

### Using yfinance

In [ ]:
# aapl_df_yf = yf.download(tickers = "aapl", start='2020-01-01', end='2020-01-31')

In [ ]:
# aapl_df_yf.head()

### Using FinRL

In FinRL's YahooDownloader, we modified the data frame to the form that convenient for further data processing process. We use adjusted close price instead of close price, and add a column representing the day of a week (0-4 corresponding to Monday-Friday).

In [ ]:
# aapl_df_finrl = YahooDownloader(start_date = '2020-01-01',
#                                 end_date = '2020-01-31',
#                                 ticker_list = ['aapl']).fetch_data()

In [ ]:
# aapl_df_finrl.head()

## Data for the chosen tickers

In [ ]:
# config_tickers.DOW_30_TICKER

In [3]:
TRAIN_START_DATE = '2007-05-30'
TRAIN_END_DATE = '2019-12-31'
TRADE_START_DATE = '2020-01-02'
TRADE_END_DATE = '2025-04-11'

In [4]:
df_raw = pd.read_csv('data.csv')

# Convert 'date' to datetime
df_raw['date'] = pd.to_datetime(df_raw['date'])

# Filter rows between TRAIN_START_DATE and TRADE_END_DATE
df_raw = df_raw[
    (df_raw['date'] >= pd.to_datetime(TRAIN_START_DATE)) &
    (df_raw['date'] <= pd.to_datetime(TRADE_END_DATE))
].copy()

df_raw['date'] = df_raw['date'].dt.strftime('%Y-%m-%d')

df_raw.tail(20)


,date,close,high,low,open,volume,tic,day
35964,2025-04-09,212.03,213.2200,190.8950,191.710,3092241.0,vb,2
35965,2025-04-09,84.55,84.9400,76.9200,78.330,10990272.0,vnq,2
35966,2025-04-09,246.69,248.4600,225.2500,226.400,2598858.0,vo,2
35967,2025-04-09,42.49,42.6600,39.8250,40.100,29071662.0,vwo,2
35968,2025-04-10,97.14,97.8600,97.1100,97.670,9940126.0,agg,3
35969,2025-04-10,91.52,91.5200,91.5100,91.510,23678142.0,bil,3
35970,2025-04-10,292.35,292.8600,287.2310,288.805,19837799.0,gld,3
35971,2025-04-10,524.58,533.4950,509.3200,532.170,162331225.0,spy,3
35972,2025-04-10,203.00,207.1800,197.4368,206.630,1781627.0,vb,3
35973,2025-04-10,82.62,84.4100,80.5000,83.420,7442686.0,vnq,3


In [5]:
df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TRADE_END_DATE,
                                ticker_list = config_tickers.DOW_30_TICKER).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


KeyboardInterrupt: 

In [6]:
df_raw.head(20)


,date,close,high,low,open,volume,tic,day
0,2007-05-30,99.380000,99.470000,99.27000,99.450000,977600.0,agg,2
1,2007-05-30,91.600000,91.600000,91.50000,91.579980,1550.0,bil,2
2,2007-05-30,64.720000,64.860000,64.60001,64.860000,2744800.0,gld,2
3,2007-05-30,153.480000,153.539990,151.34000,151.460010,131273376.0,spy,2
4,2007-05-30,75.289990,75.326510,74.23000,74.360000,35300.0,vb,2
5,2007-05-30,79.270000,79.270000,76.85001,76.850010,574700.0,vnq,2
6,2007-05-30,81.730000,81.730000,80.25999,80.259990,65600.0,vo,2
7,2007-05-30,43.500000,43.504995,42.29000,42.445000,576780.0,vwo,2
8,2007-05-31,99.160000,99.350010,99.12000,99.250000,506000.0,agg,3
9,2007-05-31,91.600000,91.600000,91.57998,91.579980,11200.0,bil,3


In [7]:
df_raw.shape

(35984, 8)

# Part 3: Preprocess Data
We need to check for missing data and do feature engineering to convert the data point into a state.
* **Adding technical indicators**. In practical trading, various information needs to be taken into account, such as historical prices, current holding shares, technical indicators, etc. Here, we demonstrate two trend-following technical indicators: MACD and RSI.
* **Adding turbulence index**. Risk-aversion reflects whether an investor prefers to protect the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the turbulence index that measures extreme fluctuation of asset price.

Hear let's take **MACD** as an example. Moving average convergence/divergence (MACD) is one of the most commonly used indicator showing bull and bear market. Its calculation is based on EMA (Exponential Moving Average indicator, measuring trend direction over a period of time.)

In [8]:
print(df_raw['tic'].unique())
print(df_raw.groupby('tic')['date'].count())


['agg' 'bil' 'gld' 'spy' 'vb' 'vnq' 'vo' 'vwo']
tic
agg    4498
bil    4498
gld    4498
spy    4498
vb     4498
vnq    4498
vo     4498
vwo    4498
Name: date, dtype: int64


In [9]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(df_raw)


Successfully added technical indicators


[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (4497, 8)
Successfully added vix
Successfully added turbulence index


In [10]:
df_raw.head(20)


,date,close,high,low,open,volume,tic,day
0,2007-05-30,99.380000,99.470000,99.27000,99.450000,977600.0,agg,2
1,2007-05-30,91.600000,91.600000,91.50000,91.579980,1550.0,bil,2
2,2007-05-30,64.720000,64.860000,64.60001,64.860000,2744800.0,gld,2
3,2007-05-30,153.480000,153.539990,151.34000,151.460010,131273376.0,spy,2
4,2007-05-30,75.289990,75.326510,74.23000,74.360000,35300.0,vb,2
5,2007-05-30,79.270000,79.270000,76.85001,76.850010,574700.0,vnq,2
6,2007-05-30,81.730000,81.730000,80.25999,80.259990,65600.0,vo,2
7,2007-05-30,43.500000,43.504995,42.29000,42.445000,576780.0,vwo,2
8,2007-05-31,99.160000,99.350010,99.12000,99.250000,506000.0,agg,3
9,2007-05-31,91.600000,91.600000,91.57998,91.579980,11200.0,bil,3


In [11]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [17]:
processed_full.head(50)

,date,tic,close,high,low,open,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2007-05-30,agg,99.380000,99.470000,99.27000,99.450000,977600.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,99.380000,99.380000,12.830000,0.0
1,2007-05-30,bil,91.600000,91.600000,91.50000,91.579980,1550.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,91.600000,91.600000,12.830000,0.0
2,2007-05-30,gld,64.720000,64.860000,64.60001,64.860000,2744800.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,64.720000,64.720000,12.830000,0.0
3,2007-05-30,spy,153.480000,153.539990,151.34000,151.460010,131273376.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,153.480000,153.480000,12.830000,0.0
4,2007-05-30,vb,75.289990,75.326510,74.23000,74.360000,35300.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,75.289990,75.289990,12.830000,0.0
5,2007-05-30,vnq,79.270000,79.270000,76.85001,76.850010,574700.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,79.270000,79.270000,12.830000,0.0
6,2007-05-30,vo,81.730000,81.730000,80.25999,80.259990,65600.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,81.730000,81.730000,12.830000,0.0
7,2007-05-30,vwo,43.500000,43.504995,42.29000,42.445000,576780.0,2.0,0.000000,99.581127,98.958873,0.000000,-66.666667,100.000000,43.500000,43.500000,12.830000,0.0
8,2007-05-31,agg,99.160000,99.350010,99.12000,99.250000,506000.0,3.0,-0.004936,99.581127,98.958873,0.000000,-66.666667,100.000000,99.270000,99.270000,13.050000,0.0
9,2007-05-31,bil,91.600000,91.600000,91.57998,91.579980,11200.0,3.0,0.000000,91.600000,91.600000,0.000000,66.666667,100.000000,91.600000,91.600000,13.050000,0.0


# Part 4: Save the Data

### Split the data for training and trading

In [15]:
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

25360
10608


### Save data to csv file

For Colab users, you can open the virtual directory in colab and manually download the files.

For users running on your local environment, the csv files should be at the same directory of this notebook.

In [16]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')